## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import requests
import time

## Data Read-in

In [2]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

1936


In [3]:
gdf = gpd.read_file('Palm_Beach_County_Boundary.geojson')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input County Name:')
county_name = input()

Input County Name:
Palm Beach County


In [5]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
445


In [6]:
print('Input total sales volume ($$$ million as 100_000_000) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million as 100_000_000) from previous (not last) month:
221_000_000


In [7]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
250_000


In [8]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
228


## Data Clean

In [9]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [10]:
df = df.dropna(subset=['SOLD DATE'])

In [11]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: October ---- Month before report month: September


In [12]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [13]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    523
Name: count, dtype: int64
-------
$/SQUARE FEET
False    523
Name: count, dtype: int64
-------
YEAR BUILT
False    523
Name: count, dtype: int64
-------


In [14]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Riviera-Beach/3100-N-Ocean-Dr-33404/unit-1109-H/home/191892887


In [15]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [16]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [17]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [27]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
239,PAST SALE,October-15-2024,Condo/Co-op,438 Saxony Unit J,Delray Beach,FL,33446.0,25000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/438-Sax...,MARMLS,A11655318,N,Y,26.440263,-80.150386
300,PAST SALE,October-29-2024,Condo/Co-op,40 Stratford Ln Unit D,Boynton Beach,FL,33436.0,45000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/40-Str...,Beaches MLS,RX-10977034,N,Y,26.485550,-80.110044
301,PAST SALE,October-24-2024,Condo/Co-op,2855 S Garden Dr #107,Lake Worth,FL,33461.0,45000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2855-Gard...,Beaches MLS,RX-10971597,N,Y,26.633633,-80.086047
276,PAST SALE,October-18-2024,Condo/Co-op,173 Flanders D #173,Delray Beach,FL,33484.0,50000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/173-Fla...,MARMLS,A11625916,N,Y,26.439640,-80.142967
302,PAST SALE,October-8-2024,Condo/Co-op,208 Suffolk E Unit 208 E,Boca Raton,FL,33434.0,52500.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/208-Suffo...,Beaches MLS,RX-10964166,N,Y,26.393076,-80.184202
237,PAST SALE,October-31-2024,Condo/Co-op,704 Flanders O,Delray Beach,FL,33484.0,65000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/704-Fla...,Beaches MLS,RX-11033054,N,Y,26.440095,-80.143513
296,PAST SALE,October-15-2024,Condo/Co-op,242 Norwich K,West Palm Beach,FL,33417.0,65000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/242-...,Beaches MLS,RX-10984180,N,Y,26.719944,-80.120176
303,PAST SALE,October-3-2024,Condo/Co-op,687 Normandy O,Delray Beach,FL,33484.0,65000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/687-Nor...,Beaches MLS,RX-10957776,N,Y,26.446145,-80.142278
288,PAST SALE,October-1-2024,Condo/Co-op,189 Norwich H,West Palm Beach,FL,33417.0,67000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/189-...,Beaches MLS,RX-10984188,N,Y,26.720040,-80.120173
287,PAST SALE,October-31-2024,Condo/Co-op,346 Flanders Dr Unit H,Delray Beach,FL,33484.0,69000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/346-Fla...,MARMLS,A11595141,N,Y,26.441343,-80.138943


In [25]:
# 277
# 307
# 290
# df_filtered = df_filtered.drop(index=[277,307,290])

In [26]:
print(df_filtered['URL'].iloc[239])

https://www.redfin.com/FL/Delray-Beach/438-Saxony-Way-33446/unit-J/home/192412693


In [27]:
# df_filtered = df_filtered.drop(index=34)

In [81]:
# # Correct the prices, if needed
# df_filtered.at[519,'PRICE']=(310_000)

# # Correct the psf, if needed
# df_filtered.at[519,'$/SQUARE FEET']=(310_000/673)

In [22]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [23]:
# print(df_filtered['URL'].iloc[291])

In [28]:
df_filtered.reset_index(inplace=True)

In [30]:
df_filtered = df_filtered.drop(columns='index')

In [31]:
df_filtered.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [32]:
print(df_filtered.iloc[44]['URL'])

https://www.redfin.com/FL/North-Palm-Beach/336-Golfview-Rd-33408/unit-401/home/42473889


In [33]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
298,45000.0,40 Stratford Ln Unit D,Boynton Beach,32.0
239,25000.0,438 Saxony Unit J,Delray Beach,35.0
276,50000.0,173 Flanders D #173,Delray Beach,54.0
299,45000.0,2855 S Garden Dr #107,Lake Worth,60.0
300,52500.0,208 Suffolk E Unit 208 E,Boca Raton,71.0
237,65000.0,704 Flanders O,Delray Beach,72.0
240,72500.0,4700 W Lucerne Lakes Blvd W #504,NaN,81.0
283,74900.0,2616 N Garden Dr #202,Lake Worth,84.0
301,65000.0,687 Normandy O,Delray Beach,86.0
285,80000.0,318 Burgundy G Unit G,Delray Beach,89.0


In [25]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [34]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [35]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [36]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [37]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [38]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [39]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip='Palm Beach County',name='Palm Beach County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [40]:
m.save('index.html')

## Summary Info

In [41]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [42]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [43]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [44]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/West-Palm-Beach/6028-Forest-Hill-Blvd-33415/unit-205/home/42157187


## Map URL Snagger

In [45]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_condo_sales_month_ending_oct_2024


## Get Summary Data

In [46]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 495
--------
Total sale price: $238,934,142
--------
Median sale price: $251,125
--------
Max sale price: $9,600,000
--------
Min sale price: $25,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,463
--------
Min price per square foot: $32
--------
Median price per square foot: $231
------------------------------------------------
CONDO AGES
Newest building: 2024.0
----------
Oldest building: 1890.0
----------
Average building age: 1985.129292929293


## Collect Agent Info

In [47]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [48]:
df_filtered.reset_index(inplace=True,drop=True)

In [49]:
max_price_index = df_filtered['PRICE'].idxmax()
min_price_index = df_filtered['PRICE'].idxmin()

max_price_psf_index = df_filtered['$/SQUARE FEET'].idxmax()
min_price_psf_index = df_filtered['$/SQUARE FEET'].idxmin()

max_year_built_index = df_filtered['YEAR BUILT'].idxmax()
min_year_built_index = df_filtered['YEAR BUILT'].idxmin()

In [50]:
index_list = [max_price_index,min_price_index,
             max_price_psf_index,min_price_psf_index,
             max_year_built_index,min_year_built_index]

In [51]:
response_list = []

for index, row in df_filtered.iterrows():
    if index in index_list:
        response_list.append(process_response(row['URL']))
        
response_df = pd.DataFrame(response_list)

merged_df = pd.merge(left=df_filtered,left_on='URL',right=response_df, right_on='url',how='outer')

In [52]:
merged_df['list_agent_name_1'].unique()

array([nan, 'Jacqueline Feldman', 'Carlos Marquez', 'Richard Ralston',
       'Julie Ann Giachetti', 'Samuel Fugina', 'Tania Cardenas'],
      dtype=object)

In [53]:
print(merged_df['URL'].iloc[0])



https://www.redfin.com/FL/Boca-Raton/100-Royal-Palm-Way-33432/unit-1080/home/42638899


In [54]:
df_filtered = merged_df

In [55]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")

Most Expensive
Boca Beach Residences, 725 S Ocean Blvd #303 Boca Raton | Price $9,600,000 | $2,218 psf | Year built: 2023 | Listing agents: Jacqueline Feldman with One Sotheby's International Realty and None with None | Buyer's agents: Lisa Bailey Levy with EXP Realty LLC and None with None
Least Expensive
KINGS POINT SAXONY CONDOS, 438 Saxony Unit J Delray Beach | Price $25,000 | $35 psf | Year built: 1974 | Listing agents: Samuel Fugina with Marlin Realty Inc and None with None | Buyer's agents: None with None and None with None
Highest Price Per Square Foot
Vizcaya Lakes Condo, 1690 Renaissance Commons Blvd #1314 Boynton Beach | Price $2,800,000 | $2,463 psf | Year built: 2007 | Listing agents: Carlos Marquez with Acme Realty and None with None | Buyer's agents: Stephanie Mordes with Aquantis Realty LLC and None with Aquantis Realty LLC
Lowest Price Per Square Foot
Stratford AT Hunters Run Condo, 40 Stratford Ln Unit D Boynton Beach | Price $45,000 | $32 psf | Year built: 1981 | Lis

## Get condo names and locations for most and least expensive

In [56]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

In [57]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    second_nut_graf_ref = 'up'
    social = 'higher'
    seo = 'increased'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    second_nut_graf_ref = 'down'
    social = 'lower'
    seo = 'dropped'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    second_nut_graf_ref = 'on par with'
    social = 'equal'
    seo = "did't change"
    
    
if subject_month_closings > prior_month_closings:
    dek_rf = 'rose'
elif prior_month_closings > subject_month_closings:
    dek_rf ='fell'
else:
    dek_rf = 'equaled'

In [58]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS', 'url',
       'list_agent_name_1', 'list_broker_name_1', 'list_agent_name_2',
       'list_broker_name_2', 'buy_agent_name_1', 'buy_broker_name_1',
       'buy_agent_name_2', 'buy_broker_name_2'],
      dtype='object')

In [59]:
# Get values for the top sale (max price)
top_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_1']
top_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_1']
top_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_2']
top_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_2']
top_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_1']
top_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_1']
top_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_2']
top_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_2']

# Get values for the bottom sale (min price)
bottom_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_1']
bottom_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_1']
bottom_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_2']
bottom_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_2']
bottom_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_1']
bottom_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_1']
bottom_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_2']
bottom_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_2']

# Now create the formatted string
test_string = f'''


TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

'''

print(test_string)





TOP SALE LISTING AGENTS:

Jacqueline Feldman with One Sotheby's International Realty and None with None

TOP SALE BUYER AGENTS:

Lisa Bailey Levy with EXP Realty LLC and None with None

BOTTOM SALE LISTING AGENTS:

Samuel Fugina with Marlin Realty Inc and None with None

BOTTOM SALE BUYER AGENTS:

None with None and None with None




In [60]:
df_filtered["PRICE"].idxmax()

78

In [61]:
story_string = f'''
\033[1mHED:\033[0m {county_name} condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county_name} {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m {county_name}’s condo sales volume {seo} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m  
\033[1mSocial:\033[0m {county_name} had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county_name}, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m

{county_name}’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, {second_nut_graf_ref} from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max():,.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max():,.0f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. Unit XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [62]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")


HED: Palm Beach County condo sales dollar volume jumps in October to $239M 
DEK: Number of closings rose to 495 from 445 in September
FEATURED HED:
SEO HED: Palm Beach County October Condo Sales Report 
SEO DESCRIPTION: Palm Beach County’s condo sales volume increased to $239 million in October.
AUTHOR: Adam Farence
RESEARCH:  
Social: Palm Beach County had $239 million in condo sales in October, higher than $221 million from September, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, monthly condo sales, condos, condo market, Boca Beach Residences, Boca Raton

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:

Palm Beach County’s NEWS PEG HERE.

October condo sales totaled $239 million, rising from $221 million in September. Brokers closed 495 sales last month, up from 445 sales in September and XXX in XXX, Mu

# Print links for notable sales

## Top sale

In [63]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Boca-Raton/725-S-Ocean-Blvd-33432/unit-303/home/188554171


## Cheapest sale

In [64]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Delray-Beach/438-Saxony-Way-33446/unit-J/home/192412693


## Highest PSF

In [65]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Boynton-Beach/1690-Renaissance-Commons-Blvd-33426/unit-1314/home/42625141


## Lowest PSF

In [66]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Boynton-Beach/40-Stratford-Ln-W-33436/unit-D/home/42330529


## Newest

In [67]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Delray-Beach/1177-George-Bush-Blvd-33483/unit-308-A/home/189018580


## Oldest

In [68]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/West-Palm-Beach/6028-Forest-Hill-Blvd-33415/unit-205/home/42157187


## Time on Market Calculator

In [70]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 8) ## List (Earlier) date
date2 = datetime(2024, 10, 15) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

37
